Этот ноутбучек создан на основе [другого ноутбучека, в котором было изложено введение в PyTorch](https://colab.research.google.com/drive/10CiI4JCAbdF6O1V-wqSzwnAzTz3SI1Ew?usp=sharing), сгенерированное при помощи Gemini от Google. Я взяла вторую часть кода - "Сеть для определения цифр из датасета MNIST". А также я взяла примеры кода из вот [этого поста с Дзена](https://dzen.ru/a/Yj2neS8AYSfP23Fl). Закоменченный код - это то, что было в том другом ноутбучеке, а сейчас не понадобится или будет задано по-другому.

Чтобы вопсользоваться таким преимуществом Lightning, как устроение вычислений при помощи графического процессора, жмякните треугольник в правом верхнем углу возле "ОЗУ/Диск", выберите "Сменить среду выполнения" и затем "T4 GPU".

In [ ]:
# Для начала установим Pytorch Lightning. Пока по дефолту в колабе его нет.
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 17.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
# Старые импорты остаявлем как есть.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Импортируем Lightning.
import pytorch_lightning as pl

In [ ]:
# Гиперпараметры оставляем как было. Но число эпох я снизила до 3.
# Трех достаточно для хорошего результата.
num_epochs = 5
batch_size = 64
learning_rate = 0.001

In [ ]:
# Загрузка данных MNIST - оставляем как было.
# Датасет MNIST берем из бибилиотеки Torchvision.
# Тренировочный датасет
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
# Тестовый датасет
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor()
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11461819.83it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 349468.87it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 1742139.50it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5427501.07it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Создание загрузчиков данных
# Ибо нельзя просто так взять и положить ваши train и test в пайторчевую модэл.
# Для загрузки используем класс DataLoader.
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
# Определение архитектуры нейросети
# class SimpleMNISTNet(nn.Module):
class SimpleMNISTNet(pl.LightningModule):

    def __init__(self):
        super(SimpleMNISTNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Входной слой
        self.fc2 = nn.Linear(128, 64)  # Скрытый слой
        self.fc3 = nn.Linear(64, 10)  # Выходной слой (10 классов)

        # Вот тут новое: добавляем функцию потерь (loss) в функцию __init__.
        self.loss_fn =  nn.CrossEntropyLoss()

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Преобразование изображения в вектор
        x = F.relu(self.fc1(x))  # Функция активации ReLU - передаем данные
        x = F.relu(self.fc2(x))  # из слоя в слой.
        x = self.fc3(x)
        return x

    # Добавляем оптимизатор. Раньше он задавался прямо в цикле обучения.
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]

    # И тоже добавляем функцию тренировки. Она тоже задавалась в цикле обучения.
    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self.forward(x)
        # Вот здесь используем добавленную в __init__ функцию потерь.
        loss = self.loss_fn(pred, y)
        self.log("train_loss", loss)
        return loss

In [ ]:
# Создание экземпляра нейросети - было так же.
model = SimpleMNISTNet()

In [ ]:
# Определение функции потерь и оптимизатора
# Оптимизацию и функцию потерь мы перенесли в класс SimpleMNISTNet,
# поэтому здесь ее убираем.
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Цикл обучения не задаем. Обучение будем проводить через Lightning.
# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         # Прямой проход
#         outputs = model(images)
#         loss = criterion(outputs, labels)

#         # Обратный проход и оптимизация
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if (i + 1) % 100 == 0:
#             print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [ ]:
# Задаем модель
trainer = pl.Trainer(max_epochs=num_epochs)
# И обучаем
trainer.fit(model, train_loader)
# Если вы переключились на ускоритель T4 GPU, то первая строчка в выводе ниже
# у вас будет такая:
# INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | fc1     | Linear           | 100 K  | train
1 | fc2     | Linear           | 8.3 K  | train
2 | fc3     | Linear           | 650    | train
3 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.438     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
# Тестирование нейросети - оставляем как было.
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Точность на тестовых данных: {100 * correct / total:.2f}%')

Точность на тестовых данных: 95.83%
